In [ ]:
import requests
import json
import pandas as pd

In [ ]:
files_endpt = "https://api.gdc.cancer.gov/files"

# The 'fields' parameter is passed as a comma-separated string of single names.
fields = [
    "file_name",
    #"analysis.workflow_type",
    #"experimental_strategy",
    "cases.project.primary_site",
    "cases.project.disease_type",
    "cases.diagnoses.primary_diagnosis",
    "cases.submitter_id",
    "cases.samples.portions.analytes.aliquots.submitter_id",
    "cases.diagnoses.tumor_stage",
    "cases.diagnoses.tumor_grade",
    "cases.diagnoses.progression_or_recurrence"
    "cases.diagnoses.vital_status",
    "cases.diagnoses.days_to_birth",
    "cases.diagnoses.days_to_death",
    "cases.diagnoses.morphology",
    "cases.diagnoses.tissue_or_organ_of_origin",
    "cases.samples.longest_dimension"
    #"cases.project.project_id"
]

fields = ','.join(fields)

In [ ]:
##CNV filters
filters = {
    "op": "and",
    "content":[
        {
        "op": "in",
        "content":{
            "field": "files.data_type",
            "value": ["Gene Level Copy Number"]
            }
        },
          {
        "op": "in",
        "content":{
               "field": "files.data_category",
                "value": ["copy number variation"]
                }
        },
        {
        "op": "in",
        "content":{
            "field": "files.data_format",
            "value": ["TXT"]
            }
        },
        {
        "op": "in",
        "content":{
            "field": "cases.project.project_id",
            "value": ['TCGA-COAD']
            }
        },
        {
        "op": "in",
        "content":{
            "field": "cases.project.program.name",
            "value": ["TCGA"]
            }
        }
        
    ]
}

In [ ]:
# A POST is used, so the filter parameters can be passed directly as a Dict object.
##remove manifest to have file fields
params = {
   "return_type": "manifest",
    "filters": filters,
    "fields": fields,
    "format": "TSV",
    "size": "15000"
    }

In [ ]:
# The parameters are passed to 'json' rather than 'params' in this case
response = requests.post(files_endpt, headers = {"Content-Type": "application/json"}, json = params)

#print(response.content.decode("utf-8"))

In [ ]:
len(response.content.decode("utf-8"))

In [ ]:
with open("manifest.txt","w") as manifest:
    manifest.write(response.content.decode("utf-8"))

In [ ]:
params = {
    "filters": json.dumps(filters),
    "fields": "primary_site,cases.project.project_id,file_name,cases.demographic.vital_status,cases.demographic.gender,cases.diagnoses.age_at_diagnosis,cases.diagnoses.days_to_last_follow_up,cases.demographic.days_to_death,cases.demographic.days_to_birth,cases.submitter_id,samples.portions.analytes.aliquots.submitter_id,cases.diagnoses.last_known_disease_status,cases.diagnoses.tumor_stage,cases.exposures.years_smoked,cases.exposures.cigarettes_per_day,cases.samples.portions.analytes.aliquots.submitter_id",
    "format": "TSV",
    "size": "50000"
    }
response = requests.get("https://api.gdc.cancer.gov/files", headers = {"Content-Type": "application/json"}, params = params)
#print(response.content.decode("UTF-8"))
with open("files.txt","w") as files:
    files.write(response.content.decode("utf-8"))

In [ ]:
df_files = pd.read_csv("files.txt", sep='\t')
df_files.drop("id", axis=1, inplace=True)
df_files.set_index("file_name", inplace=True)
df_files.head(2)

In [ ]:
df_files.to_csv("files_cnv.dat", header=True)